In [4]:
%run default-imports.ipynb

In [5]:
def run_experiment(params):
    
    print("Running experiment with following parameters: ")
    print(params)
    
    filenames = {'MIMIC' : "~/cohorts/hs_mimic.csv", 'SINAI' : "~/cohorts/hs_sinai_preprocessed.csv"}

    ''' first load and split data '''
    data = Load().execute(filename=filenames[params['cohort']])
    train, test = Split().execute(data,test_size=params['test_size'])
    train = Impute().execute(train.copy(), imputation_method=params['imputation_method'])
    
    if params.get('transform_method'):
        train = Scale().execute(train.copy(), params['target'],transform_method=params['transform_method'])
    
    if params.get('sampling_method'):
        train = Sample().execute(train.copy(), params['target'],sampling_method=params['sampling_method'])        
    
    ''' fit models '''
    models = Train().execute(train, target=params['target'], algorithms=[params['algorithm']], optimize=params['optimize_mode'])

    ''' evaluate models '''
    test = Impute().execute(test.copy(), imputation_method=params['imputation_method'])
    if params.get('transform_method'):
        test = Scale().execute(test.copy(), params['target'],transform_method=params['transform_method'])
    
    results = Evaluate().execute(test, target=params['target'], models=models)
    
    ''' obtain performance metrics '''
    performance = {}        
    performance['discrimination'] = get_discrimination_metrics(**results[params['algorithm']])     
    performance['calibration'] = get_calibration_metrics(results[params['algorithm']]['y_true'],results[params['algorithm']]['y_probs'])
    performance['clinical_usefulness'] = get_clinical_usefulness_metrics(performance['discrimination'])
    
    ''' interpret explanations '''
    explanations = {}     
    weighted_explanations = {}
    if params.get('explainers'):
        explanations = Explain().execute(train, models=models, target=params['target'], explainers=params['explainers'], exp_kwargs={'test':test})        
        weighted_explanations = get_weighted_explanations(explanations[params['algorithm']])
        
    ''' store results '''
    experiment = {'parameters' : params, 'performance' : performance, 'explanations' : explanations, 'weighted_explanations' : weighted_explanations}
    
    return experiment 
    
    